<a href="https://colab.research.google.com/github/vincentmartin/tp-fine-tuning-student-version/blob/main/tp-fine-tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP fine-tuning de LLM

Dans ce notebook vous allez fine tuner un LLM de base, Flan T5, avec la technique PEFT et LoRA.

### Instruction à suivre pour exécution sur Google Colab

Aller dans `Execution -> Modifier le type d'exécution` puis sélectionner `T4-GPU` pour exploiter les fonctionnalités GPU.

![Colab GPU](resources/colab_gpu.png "T4-GPU")

Installationd des dépendances

In [38]:
%pip install -U datasets

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch \
    torchdata --quiet

%pip install \
    transformers \
    evaluate \
    rouge_score \
    loralib \
    peft \
    bitsandbytes

Import des dépendances

In [39]:
from datasets import load_dataset
from transformers import AutoModel, AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, BitsAndBytesConfig
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import os
import bitsandbytes
os.environ["WANDB_DISABLED"] = "true"

Chargement du LLM de base.

In [40]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Création d'une fonction pour afficher le nombre de paramètres entraînables.

In [41]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


## Fine tuning avec PEFT et LoRA

Le fine tuning complet d'un modèle n'est pas un choix judicieux pour un particulier ou une entreprise qui n'a pas une énorme puissance de calcul. La méthode la plus appropriée est d'utiliser PEFT (_Parameter Efficient Fine-Tuning_).

PEFT est un ensemble de technique qui incluant LORA (_Low Rank Adaptation_) et le _prompt tuning_ (**différent du prompt engineering**). LORA permet de fine tuner un modèle avec peu de ressources matérielles (un ou deux GPU). LORA permet de créer des adapteurs composés de 1-10% des paramètres du LLM original. De plus, le LLM original n'est pas modifié, ce qui permet de rapidement changer d'adapteurs en fonction du cas d'usage.

### Configuration de PEFT / LoRA

Premièrement, configurons PEFT/LoRA pour fine tuner notre modèle de base avec ce que l'on appelle _adapteur_.

PEFT/LoRA gêle les couches du LLM original pour n'entraîner que l'adapteur.

In [42]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank : plus il est grand, plus il y a de paramètres. Idéal : 16-32
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # Pour FLANT5, laisser ce type
)

Ajouter l'adapteur au LLM original.

In [43]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%


## Lancement de l'entraînement

Chargeons le jeu de données pour l'entraînement.

In [44]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Pour que l'entraînement prenne un temps acceptable dans ce notebook, nous diminuons la taille du jeu de données.

In [45]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

**Exercice**  : en vous aidant de la documentation https://huggingface.co/docs/transformers/v4.15.0/en/main_classes/trainer#transformers.TrainingArguments, créer une instance de **Trainer** pour entraîner le LLM. Vous utiliserez les paramètres suivants :
- auto_find_batch_size=True,
- learning_rate=1e-3,
- num_train_epochs=5,
- logging_steps=1,
- max_steps=1   

Le jeu de données à utiliser pour l'entraînement est `tokenized_datasets["train"]`.

**Dans Google Colab, utiliser `report_to=None` sinon il vous sera demandé une clef Wanadb.**

In [46]:
output_dir = './training-output'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=2,
    logging_steps=1,
    max_steps=-1,
    report_to=None
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


**Exercice** : Lancer l'entraînement et sauvegarder le modèle (adapteur)  et le tokenizer dans le dossier `training-output-checkpoint`.

In [47]:
peft_trainer.train()


Step,Training Loss
1,48.000000
2,46.250000
3,42.250000
4,38.500000
5,33.500000
6,29.500000
7,27.375000
8,26.250000
9,24.000000
10,22.125000


TrainOutput(global_step=32, training_loss=15.76171875, metrics={'train_runtime': 75.996, 'train_samples_per_second': 3.29, 'train_steps_per_second': 0.421, 'total_flos': 173907247104000.0, 'train_loss': 15.76171875, 'epoch': 2.0})

In [48]:
output_dir = "./training-output-checkpoint"

# Sauvegarde de l'adapteur LoRA
peft_model.save_pretrained(output_dir)

# Sauvegarde du tokenizer
tokenizer.save_pretrained(output_dir)


('./training-output-checkpoint/tokenizer_config.json',
 './training-output-checkpoint/special_tokens_map.json',
 './training-output-checkpoint/spiece.model',
 './training-output-checkpoint/added_tokens.json',
 './training-output-checkpoint/tokenizer.json')

### Evaluation du modèle fine tuné

Une erreur classique lorsque l'on début est d'évaluer les performances en 'regardant' quelques générations manuellement. C'est une mauvaise idée car (1) ce n'est pas quantifié et (2) ce qui fonctionne sur quelques exemples ne fonctionne peut être pas sur des milliers d'exemples (principe de généralisation).

Lorsque l'on fine tune un modèle, il est donc capital de mesurer les performances pour savoir si **globalement** les résultats sont meilleurs.

In [49]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       'training-output-checkpoint',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)


In [50]:
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

device = "cuda" if torch.cuda.is_available() else "cpu"


prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

original_model_outputs = original_model.to(device).generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)


peft_model_outputs = peft_model.to(device).generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'RESUME HUMAIN:\n{human_baseline_summary}')
print(dash_line)
print(f'RESUME AVEC MODELE ORIGINAL:\n{original_model_text_output}')
print(dash_line)
print(dash_line)
print(f'RESUME AVEC MODELE PEFT: {peft_model_text_output}')

---------------------------------------------------------------------------------------------------
RESUME HUMAIN:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
RESUME AVEC MODELE ORIGINAL:
You may want to add a painting program to your software.
---------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------
RESUME AVEC MODELE PEFT: Upgrade your system.


Inférence sur 10 exemples du jeu de test.

In [51]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    human_baseline_text_output = human_baseline_summaries[idx]

    original_model_outputs = original_model.to(device).generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.to(device).generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])

**Exercice** : en utilisant la documentation https://huggingface.co/docs/evaluate/main/en/choosing_a_metric, calculer le score ROUGE entre :
- Les résumés du modèle original  (`original_model_summaries`)  vs. résumés humain (`human_baseline_summaries`).
- Les résumés du modèle peft  (`peft_model_summaries`) vs. résumé humain (`human_baseline_summaries`).

Afficher les scores et commentez les.

In [52]:
import evaluate

rouge = evaluate.load("rouge")

rouge_original = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries,
    use_stemmer=True
)

rouge_peft = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries,
    use_stemmer=True
)

print("ROUGE - Modèle original vs Humain")
print(rouge_original)
print("\nROUGE - Modèle PEFT vs Humain")
print(rouge_peft)


ROUGE - Modèle original vs Humain
{'rouge1': np.float64(0.19352324882854738), 'rouge2': np.float64(0.05302298345776606), 'rougeL': np.float64(0.1607905803195963), 'rougeLsum': np.float64(0.1635356842484681)}

ROUGE - Modèle PEFT vs Humain
{'rouge1': np.float64(0.19899491249491247), 'rouge2': np.float64(0.08550724637681159), 'rougeL': np.float64(0.1704131054131054), 'rougeLsum': np.float64(0.17578551078551077)}


**Exercice** : calculer le gain de performance en pourcentage du modèle PEFT sur le modèle original

In [53]:
# Calcul du gain de performance en pourcentage (PEFT vs Original)

gain_percent = {}

for metric in rouge_original.keys():
    original_score = rouge_original[metric]
    peft_score = rouge_peft[metric]

    gain_percent[metric] = ((peft_score - original_score) / original_score) * 100

print("Gain de performance du modèle PEFT par rapport au modèle original (%)\n")
for metric, gain in gain_percent.items():
    print(f"{metric}: {gain:.2f} %")


Gain de performance du modèle PEFT par rapport au modèle original (%)

rouge1: 2.83 %
rouge2: 61.26 %
rougeL: 5.98 %
rougeLsum: 7.49 %


# Rapport d’évaluation ROUGE – Modèle Original vs Modèle PEFT (entraînement prolongé)

## 1. Contexte
Cette évaluation compare les performances du modèle **Flan-T5 base** :
- **Modèle original** (sans fine-tuning),
- **Modèle fine-tuné avec PEFT / LoRA** (entraînement prolongé),

sur **10 exemples du jeu de test DialogSum**, à l’aide de la métrique **ROUGE**, utilisée pour évaluer la qualité des résumés automatiques par comparaison aux **résumés humains de référence**.

---

## 2. Résultats quantitatifs

| Métrique     | Modèle original | Modèle PEFT | Δ absolu (PEFT − Original) | Gain relatif |
|--------------|-----------------|-------------|----------------------------|--------------|
| ROUGE-1      | 0.1935          | 0.1990      | +0.00547                   | +2.83 %      |
| ROUGE-2      | 0.0530          | 0.0855      | +0.03248                   | +61.26 %     |
| ROUGE-L      | 0.1608          | 0.1704      | +0.00962                   | +5.98 %      |
| ROUGE-Lsum   | 0.1635          | 0.1758      | +0.01225                   | +7.49 %      |

---

## 3. Analyse détaillée

### 3.1 ROUGE-1 – Couverture lexicale
Le score **ROUGE-1** progresse légèrement (+2.83 %), indiquant une amélioration modeste mais cohérente dans la sélection des **mots clés** par le modèle PEFT.  
Cela suggère que le fine-tuning n’altère pas fortement le vocabulaire, mais affine l’identification des termes pertinents.

---

### 3.2 ROUGE-2 – Cohérence locale (attention au pourcentage)
Le gain relatif sur **ROUGE-2** (+61.26 %) peut sembler très élevé, mais il doit être interprété avec précaution :

- le score initial est **très faible** (≈ 0.053),
- le gain absolu réel est de **+0.032**, ce qui reste modéré,
- ROUGE-2 est **très sensible** aux variations, surtout sur un **échantillon réduit (10 exemples)**.

👉 Ce résultat indique une **meilleure cohérence locale** et de meilleurs enchaînements de mots.

---

### 3.3 ROUGE-L et ROUGE-Lsum – Structure globale
Les métriques **ROUGE-L (+5.98 %) et ROUGE-Lsum (+7.49 %)** montrent une amélioration plus stable et significative :

- meilleure organisation globale du résumé,
- ordre des informations plus proche des résumés humains.

Ces métriques sont généralement plus fiables que ROUGE-2 sur de petits échantillons.

---

## 4. Interprétation globale

- Le modèle **PEFT surpasse systématiquement le modèle original** sur toutes les métriques ROUGE.
- Les améliorations les plus fiables concernent :
  - la **structure du résumé** (ROUGE-L / Lsum),
  - la **cohérence locale** (ROUGE-2, à confirmer à plus grande échelle).
- Les gains sont obtenus avec seulement **~1–2 % de paramètres entraînables**, illustrant l’efficacité de PEFT / LoRA.

---

## 5. Conclusion

L’entraînement prolongé du modèle PEFT conduit à une **amélioration cohérente et mesurable** par rapport au modèle original, en particulier sur la **structure et la cohérence des résumés**.

Ces résultats confirment que **PEFT / LoRA est une approche efficace et économiquement viable** pour spécialiser un LLM sur une tâche donnée.  
Une évaluation sur un jeu de test plus large permettrait de consolider ces conclusions et de réduire la variance observée.


## Fine tuning de Llama 3 ou Qwen 3 1.7B

Le modèle `flan-t5-base`que nous avons utilisé jusqu'à maintenant est bien pour comprendre les principes mais c'est un modèle ancien aux performances dépassées par rapport aux modèles récents tels que Llama 3.

Dans cet exercice, vous allez charger puis fine tuner un LLM bien plus performant tout en conservant une taille acceptable de 3B de paramètres : Llama 3.2 - 3B. Nous pouvons aussi tester avec Qwen 3 1.7B (https://huggingface.co/Qwen/Qwen3-1.7B).

Afin que le modèle puisse être chargé en VRAM, nous utiliserons une version quantisée en 4bits : https://huggingface.co/unsloth/Llama-3.2-3B-Instruct-bnb-4bit. L'utilisation de la bibliothèque `bitsandbytes`est alors indispensable.

**Redémarrer la session à ce stade pour réinitialiser la RAM et la VRAM**

### Conseils pour réaliser l'exercice :

- Le modèle n'est plus de type _Encoder Decoder_ (Seq2Seq) mais _Decoder only_ (CausalLM). Effectuer les modifications en conséquence
- Réduire la taille du jeu de données d'entraînement pour rester dans des temps acceptables (100 exemples)
- Modifier les arguments d'entraînement (`TrainingArguments`) pour prendre accélérer le traitement : considérer les paramètres `per_device_train_batch_size`, `gradient_accumulation_steps`, `gradient_chekpointing`.

L'exercice peut prendre un certain temps, faites votre maximum et avancer pas à pas.

In [1]:
%pip install -U datasets

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch \
    torchdata --quiet

%pip install \
    transformers \
    evaluate \
    rouge_score \
    loralib \
    peft \
    bitsandbytes

In [2]:
from datasets import load_dataset
from transformers import AutoModel, AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, BitsAndBytesConfig
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import os
import bitsandbytes
os.environ["WANDB_DISABLED"] = "true"



In [3]:
model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

original_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)
original_model.config.use_cache = False
print("OK loaded:", model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


OK loaded: unsloth/Llama-3.2-3B-Instruct-bnb-4bit


In [4]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 394177536
all model parameters: 1803463680
percentage of trainable model parameters: 21.86%


In [5]:
from peft import LoraConfig, get_peft_model, TaskType
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

In [6]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 18350080
all model parameters: 1821813760
percentage of trainable model parameters: 1.01%


In [8]:
from datasets import DatasetDict

huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

dataset_small = DatasetDict({
    "train": dataset["train"].select(range(100)),
    "validation": dataset["validation"].select(range(50)),
    "test": dataset["test"].select(range(50)),
})

max_length = 384  # baisse à 256 si besoin

def tokenize_function(example):
    start_prompt = "Summarize the following conversation.\n\n"
    end_prompt = "\n\nSummary: "

    prompts = [start_prompt + d + end_prompt for d in example["dialogue"]]
    summaries = [s.strip() + tokenizer.eos_token for s in example["summary"]]

    input_ids_batch, attention_masks_batch, labels_batch = [], [], []

    for prompt, summary in zip(prompts, summaries):
        prompt_ids = tokenizer(prompt, add_special_tokens=False).input_ids
        summary_ids = tokenizer(summary, add_special_tokens=False).input_ids

        input_ids = (prompt_ids + summary_ids)[:max_length]
        labels = ([-100] * len(prompt_ids) + summary_ids)[:max_length]
        attention_mask = [1] * len(input_ids)

        pad_len = max_length - len(input_ids)
        if pad_len > 0:
            input_ids += [tokenizer.pad_token_id] * pad_len
            attention_mask += [0] * pad_len
            labels += [-100] * pad_len

        input_ids_batch.append(input_ids)
        attention_masks_batch.append(attention_mask)
        labels_batch.append(labels)

    return {
        "input_ids": input_ids_batch,
        "attention_mask": attention_masks_batch,
        "labels": labels_batch,
    }

tokenized_datasets = dataset_small.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["id", "topic", "dialogue", "summary"])


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [9]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/50 [00:00<?, ? examples/s]

Filter:   0%|          | 0/50 [00:00<?, ? examples/s]

In [10]:
output_dir = './training-output'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=2,
    logging_steps=1,
    max_steps=-1,
    report_to=None
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [11]:
peft_trainer.train()


Step,Training Loss
1,2.547400
2,1.403500


TrainOutput(global_step=2, training_loss=1.9754880666732788, metrics={'train_runtime': 13.1822, 'train_samples_per_second': 0.152, 'train_steps_per_second': 0.152, 'total_flos': 13073345150976.0, 'train_loss': 1.9754880666732788, 'epoch': 2.0})

In [12]:
output_dir = "./training-output-checkpoint"

# Sauvegarde de l'adapteur LoRA
peft_model.save_pretrained(output_dir)

# Sauvegarde du tokenizer
tokenizer.save_pretrained(output_dir)


('./training-output-checkpoint/tokenizer_config.json',
 './training-output-checkpoint/special_tokens_map.json',
 './training-output-checkpoint/chat_template.jinja',
 './training-output-checkpoint/tokenizer.json')

In [14]:
from peft import PeftModel

adapter_dir = "training-output-checkpoint"

peft_model = PeftModel.from_pretrained(
    original_model,
    adapter_dir,
    is_trainable=False
)
peft_model.eval()
original_model.eval()


/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=3072, out_features=32, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=32, out_features=3072, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=3072, out_features=1024, bias=False)
            (lora_dropout)

In [15]:
index = 0  # dataset_small["test"] n'a que 50 exemples -> mets 0..49
dialogue = dataset_small["test"][index]["dialogue"]
human_baseline_summary = dataset_small["test"][index]["summary"]

device = "cuda" if torch.cuda.is_available() else "cpu"

prompt = f"Summarize the following conversation.\n\n{dialogue}\n\nSummary: "
inputs = tokenizer(prompt, return_tensors="pt").to(peft_model.device)

gen_cfg = GenerationConfig(max_new_tokens=120, num_beams=1)

with torch.no_grad():
    out_orig = original_model.generate(**inputs, generation_config=gen_cfg)
    out_peft = peft_model.generate(**inputs, generation_config=gen_cfg)

prompt_len = inputs["input_ids"].shape[1]
original_text = tokenizer.decode(out_orig[0][prompt_len:], skip_special_tokens=True).strip()
peft_text = tokenizer.decode(out_peft[0][prompt_len:], skip_special_tokens=True).strip()

dash_line = "-" * 100
print(dash_line)
print(f"RESUME HUMAIN:\n{human_baseline_summary}")
print(dash_line)
print(f"RESUME AVEC MODELE ORIGINAL:\n{original_text}")
print(dash_line)
print(f"RESUME AVEC MODELE PEFT:\n{peft_text}")
print(dash_line)


`generation_config` default values have been modified to match model-specific defaults: {'max_length': 131072, 'do_sample': True, 'temperature': 0.6, 'top_p': 0.9, 'pad_token_id': 128004, 'bos_token_id': 128000, 'eos_token_id': [128001, 128008, 128009]}. If this is not desired, please set these values explicitly.


----------------------------------------------------------------------------------------------------
RESUME HUMAIN:
Ms. Dawson helps #Person1# to write a memo to inform every employee that they have to change the communication method and should not use Instant Messaging anymore.
----------------------------------------------------------------------------------------------------
RESUME AVEC MODELE ORIGINAL:
1) Ms. Dawson needs a dictation, 2) A new policy will be implemented, 3) All office communications are restricted to email and official memos, 4) Instant Messaging is prohibited, 5) Employees may need to change their communication methods, 6) Department heads will handle any questions.
----------------------------------------------------------------------------------------------------
RESUME AVEC MODELE PEFT:
1) Ms. Dawson is asked to take a dictation for a memo. 2) The memo will be sent to all employees by this afternoon. 3) All office communications are restricted to email and offi

In [16]:
dialogues = dataset_small["test"][0:10]["dialogue"]
human_baseline_summaries = dataset_small["test"][0:10]["summary"]

original_model_summaries = []
peft_model_summaries = []

gen_cfg = GenerationConfig(max_new_tokens=120, num_beams=1)

for dialogue in dialogues:
    prompt = f"Summarize the following conversation.\n\n{dialogue}\n\nSummary: "
    inputs = tokenizer(prompt, return_tensors="pt").to(peft_model.device)
    prompt_len = inputs["input_ids"].shape[1]

    with torch.no_grad():
        out_orig = original_model.generate(**inputs, generation_config=gen_cfg)
        out_peft = peft_model.generate(**inputs, generation_config=gen_cfg)

    original_text = tokenizer.decode(out_orig[0][prompt_len:], skip_special_tokens=True).strip()
    peft_text = tokenizer.decode(out_peft[0][prompt_len:], skip_special_tokens=True).strip()

    original_model_summaries.append(original_text)
    peft_model_summaries.append(peft_text)

df = pd.DataFrame(
    list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries)),
    columns=["human_baseline_summaries", "original_model_summaries", "peft_model_summaries"]
)
df


,human_baseline_summaries,original_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,Dawson instructs Person2 to take a dictation a...,The conversation is about creating an intra-of...
1,In order to prevent employees from wasting tim...,Person1 instructs Person2 to write a memo to a...,The conversation is between Person1 (Ms. Dawso...
2,Ms. Dawson takes a dictation for #Person1# abo...,1. All office communications are restricted to...,Person1 instructs Person2 to take dictation an...
3,#Person2# arrives late because of traffic jam....,#Person1 and #Person2 are discussing #Person2'...,2 people have a conversation about one person'...
4,#Person2# decides to follow #Person1#'s sugges...,Person1 advises Person2 to take the public tra...,2 people are discussing the issue of traffic a...
5,#Person2# complains to #Person1# about the tra...,1. Person2 is late due to traffic. 2. Person1 ...,Person 1 and Person 2 are having a conversatio...
6,#Person1# tells Kate that Masha and Hero get d...,"2 people, Person1 and Person2, discussing the ...",2 people are discussing a surprising news abou...
7,#Person1# tells Kate that Masha and Hero are g...,2 people are discussing a surprising divorce b...,2 people are discussing a surprising news: Kat...
8,#Person1# and Kate talk about the divorce betw...,2 people discussing a surprising news about a ...,2 people are discussing a surprising news that...
9,#Person1# and Brian are at the birthday party ...,A person is celebrating their birthday and is ...,"2 people, Person1 and Person2, are celebrating..."


In [17]:
import evaluate
rouge = evaluate.load("rouge")

rouge_original = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries,
    use_stemmer=True
)

rouge_peft = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries,
    use_stemmer=True
)

print("ROUGE - Modèle original vs Humain")
print(rouge_original)
print("\nROUGE - Modèle PEFT vs Humain")
print(rouge_peft)


ROUGE - Modèle original vs Humain
{'rouge1': np.float64(0.26759568383220966), 'rouge2': np.float64(0.07767939133688473), 'rougeL': np.float64(0.18736750895432255), 'rougeLsum': np.float64(0.1869093374006663)}

ROUGE - Modèle PEFT vs Humain
{'rouge1': np.float64(0.2558258207486179), 'rouge2': np.float64(0.08433112018811738), 'rougeL': np.float64(0.1921823771346873), 'rougeLsum': np.float64(0.19223927954848757)}


In [20]:
# Calcul du gain de performance en pourcentage (PEFT vs Original)

gain_percent = {}

for metric in rouge_original.keys():
    original_score = rouge_original[metric]
    peft_score = rouge_peft[metric]

    gain_percent[metric] = ((peft_score - original_score) / original_score) * 100

print("Gain de performance du modèle PEFT par rapport au modèle original (%)\n")
for metric, gain in gain_percent.items():
    print(f"{metric}: {gain:.2f} %")


Gain de performance du modèle PEFT par rapport au modèle original (%)

rouge1: -4.40 %
rouge2: 8.56 %
rougeL: 2.57 %
rougeLsum: 2.85 %


# Rapport d’évaluation ROUGE – Llama 3.2-3B (Original vs PEFT / LoRA)

## 1. Contexte
Cette évaluation compare les performances du modèle **Llama 3.2-3B Instruct** :
- **Modèle original** (quantifié 4-bit, sans fine-tuning),
- **Modèle fine-tuné avec PEFT / LoRA**,

sur **10 exemples du jeu de test DialogSum**, à l’aide de la métrique **ROUGE**, standard pour l’évaluation automatique de résumés.

Le passage de Flan-T5 à Llama 3.2 implique un changement de paradigme (modèle **decoder-only / CausalLM**), ce qui rend l’analyse de la structure et de la cohérence encore plus importante.

---

## 2. Résultats quantitatifs

| Métrique     | Modèle original | Modèle PEFT | Δ absolu (PEFT − Original) | Gain relatif |
|--------------|-----------------|-------------|----------------------------|--------------|
| ROUGE-1      | 0.2676          | 0.2558      | −0.0118                    | −4.4 %       |
| ROUGE-2      | 0.0777          | 0.0843      | +0.0067                    | +8.6 %       |
| ROUGE-L      | 0.1874          | 0.1922      | +0.0048                    | +2.6 %       |
| ROUGE-Lsum   | 0.1869          | 0.1922      | +0.0053                    | +2.8 %       |

---

## 3. Analyse détaillée

### 3.1 ROUGE-1 – Couverture lexicale
Le score **ROUGE-1 diminue légèrement** pour le modèle PEFT (−4.4 %).  
Cela indique que le fine-tuning LoRA n’augmente pas la quantité de mots partagés avec les résumés humains, et peut même introduire une formulation plus libre ou plus abstraite.

---

### 3.2 ROUGE-2 – Cohérence locale
Le score **ROUGE-2 augmente de manière nette (+8.6 %)**.  
Cela suggère une amélioration des **enchaînements de mots (bigrams)** et de la cohérence locale des phrases générées par le modèle PEFT.

C’est un signal positif indiquant que le fine-tuning aide le modèle à produire des résumés plus structurés linguistiquement.

---

### 3.3 ROUGE-L et ROUGE-Lsum – Structure globale
Les métriques **ROUGE-L (+2.6 %) et ROUGE-Lsum (+2.8 %)** progressent légèrement.

Ces scores mesurent la similarité de la **structure globale** et de l’ordre des informations par rapport aux résumés humains.  
L’amélioration suggère que le modèle PEFT :
- organise mieux les informations clés,
- suit une structure plus proche des références humaines.

---

## 4. Interprétation globale

- Le fine-tuning PEFT **n’améliore pas toutes les métriques uniformément**.
- Les gains se concentrent principalement sur :
  - la **cohérence locale** (ROUGE-2),
  - la **structure globale** du résumé (ROUGE-L / Lsum).
- La légère baisse de ROUGE-1 est cohérente avec un modèle plus **abstrait** et moins focalisé sur le recouvrement mot-à-mot.

---

## 5. Conclusion

Les résultats montrent que le fine-tuning **PEFT / LoRA sur Llama 3.2-3B** apporte une **amélioration qualitative ciblée**, principalement sur la **cohérence et la structure** des résumés, au prix d’une légère baisse du recouvrement lexical.

Cela confirme que **PEFT est pertinent même pour des modèles récents et puissants**, en permettant une adaptation fine au cas d’usage tout en conservant un coût computationnel très réduit.

👉 Une évaluation sur un jeu de test plus large permettrait de confirmer ces tendances et d’évaluer la généralisation du modèle.
